In [1]:
import os, sys
import glob
import pandas as pd
import json


PROOT=os.getcwd()

if not os.path.exists("dalton"):
    os.mkdir("dalton")
from madnessToDaltony import *
madmol_files=glob.glob("*.mol")
with open(PROOT+'/molecules/frequency.json') as json_file:
        freq_json = json.loads(json_file.read())

/home/adrianhurtado/testing/madness-test-suite/madness-test-suite


In [2]:
def dalton_excited_input(madmol, xc, basis,num_states):
    molname = madmol.split('.')[0]
    dalton_inp = []
    dalton_inp.append('**DALTON INPUT')
    dalton_inp.append('.RUN PROPERTIES')
    dalton_inp.append('**WAVE FUNCTIONS')
    if xc == 'hf':
        dalton_inp.append('.HF')
    else:
        dalton_inp.append('.DFT')
        dalton_inp.append(xc.capitalize())
    dalton_inp.append('**PROPERTIES')
    dalton_inp.append('.EXCITA')
    dalton_inp.append('*EXCITA')
    dalton_inp.append('.NEXCIT')
    
    states=[]
    for i in range(10):
        states.append(str(num_states))
    dalton_inp.append('   '+'  '.join(states))
        

    dalton_inp.append('**END OF DALTON INPUT')
    dalton_inp = '\n'.join(dalton_inp)
    run_dir = PROOT+"/dalton/"+xc+'/'+molname+'/'+'excited_state'
    if not os.path.exists(run_dir):
        os.makedirs(run_dir)
    madmolfile = PROOT+'/molecules/'+madmol+".mol"
    mol_input = madmol_to_dalmol(madmolfile, basis)
    dal_run_file = run_dir+'/excited.dal'
    with open(dal_run_file, 'w') as file:  # Use file to refer to the file object
        file.write(dalton_inp)
    mol_file = run_dir+"/"+molname+'-'+basis.replace('*', 'S')+'.mol'
    with open(mol_file, 'w') as file:  # Use file to refer to the file object
        file.write(mol_input)
    return run_dir, dal_run_file.split('/')[-1], mol_file.split('/')[-1]

madmol='Be'
xc='hf'
num_states=freq_json[madmol][xc]['excited_state']
run_dir,dal_run_file,mol_file=dalton_excited_input(madmol,xc,'aug-cc-pVDZ',num_states)

In [6]:
from daltonToJson import daltonToJson
madmol='HCl'
xc='hf'
operator='excited_state'
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
def basis_set_calculation(madmol,xc,operator,basis_list):
    output_files=[]
    num_states=freq_json[madmol][xc]['excited_state']
    for basis in basis_list:
        run_directory,dal_input,mol_input=dalton_excited_input(madmol,xc,basis,num_states)
        d_output_file=dal_input.split('.')[0]+"_"+mol_input.split('.')[0]+'.out'
        d_out,d_error=run_dalton(run_directory,dal_input,mol_input)
        with open(run_directory+'/'+d_output_file) as outfile:
            json_obj=daltonToJson()
            output_files.append(json_obj.convert(outfile))
        print(output_files[-1])
            
            
    return output_files
        
        
outfiles=basis_set_calculation(madmol,xc,operator,basis_list)

a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.016       0.016

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Singlet electronic excitation energies
clearing no good if seen more than twice
yoou founnd meeeee
                 Sym.   Mode   Frequency    Frequency

                 Sym.   Mode   Frequency    Frequency

                ex. st.  No.      (au)         (eV)

               ----------------------------------------

found ------ appending 
read line: ['1', '1', '0.414264', '11.272702']
read line: ['1', '2', '0.430215', '11.706740']
read line: ['1', '3', '0.471445', '12.828682']
read line: ['1', '4', '0.510283', '13.885514']
read line: ['2', '1', '0.305791', '8.320990']
read line: ['2', '2', '0.385584', '10.492270']
read line: ['2', '3', '0

In [44]:
import pandas
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
import numpy as np
#madmol='HCl'
#xc='hf'
#operator='dipole'
# create an energy table

def createExcitedTables(outfiles,basis_list,num_states):
    # generate tables given name of output files and basis_list used to generate output files
    
    data={'totalEnergy':[],'nuclearRepulsionEnergy':[],'electronEnergy':[]}
    rdata={"sym":[],"mode":[],'freq':[]}
    r_dict={}
    s_dict={}
    totals=[]
    for basis,outfile in zip(basis_list,outfiles):
        e_data=json.loads(outfile)['simulation']['calculations'][0]
        r_data=json.loads(outfile)['simulation']['calculations'][1]
        for dkeys in data.keys():
            data[dkeys].append(float(e_data['calculationResults']
                                     [dkeys]['value']))
        freq=r_data['calculationResults']['frequencies']
        freq=[f for l in freq for f in l]
        freq=np.array(freq)
        sort_f=np.argsort(freq)
        freq=freq[sort_f]
        print(sort_f)
        
        Mode=r_data['calculationResults']['Mode']
        Mode=[m for mo in Mode for m in mo]
        Mode=np.array(Mode)
        Mode=Mode[sort_f]
        Sym=r_data['calculationResults']['Sym']
        Sym=[s for so in Sym for s in so]
        Sym=np.array(Sym)
        Sym=Sym[sort_f]
        
        print(freq)
        print(Mode)
        print(Sym)
        print(r_dict)
        r_dict['sym']=Sym.tolist()   
        r_dict['mode']=Mode.tolist()  
        r_dict[basis]=freq.tolist()    
        rf=pd.DataFrame.from_dict(r_dict)
        
        s_dict['Sym']=Sym.tolist()   
        s_dict['Mode']=Mode.tolist()  
        s_dict['freq']=freq.tolist()  
        totals.append(pd.DataFrame.from_dict(s_dict))
        
        
    df=pd.DataFrame.from_dict(data,orient='columns')
    return df.T,rf,totals

energy_table,response_table,full_table=createExcitedTables(outfiles,basis_list,4)
energy_table.to_csv()
print(energy_table)
print(response_table)


[ 4  8  5  9  6 10  0 12 13  1  2  7 11  3 14 15]
[0.305791 0.305791 0.385584 0.385584 0.407688 0.407688 0.414264 0.414264
 0.417174 0.430215 0.471445 0.48246  0.48246  0.510283 0.510283 0.510843]
[1 1 2 2 3 3 1 1 2 2 3 4 4 4 3 4]
[2 3 2 3 2 3 1 4 4 1 1 2 3 1 4 4]
{}
[ 4  8  5  9  6 10  0 12 13  1  2  7 11  3 14 15]
[0.304283 0.304283 0.374196 0.374196 0.400015 0.400015 0.405311 0.405311
 0.408506 0.419503 0.461998 0.465189 0.465189 0.465644 0.465644 0.467806]
[1 1 2 2 3 3 1 1 2 2 3 4 4 4 3 4]
[2 3 2 3 2 3 1 4 4 1 1 2 3 1 4 4]
{'sym': [2, 3, 2, 3, 2, 3, 1, 4, 4, 1, 1, 2, 3, 1, 4, 4], 'mode': [1, 1, 2, 2, 3, 3, 1, 1, 2, 2, 3, 4, 4, 4, 3, 4], 'aug-cc-pVDZ': [0.305791, 0.305791, 0.385584, 0.385584, 0.407688, 0.407688, 0.414264, 0.414264, 0.417174, 0.430215, 0.471445, 0.48246, 0.48246, 0.510283, 0.510283, 0.510843]}
[ 4  8  5  9  6 10  0 12 13  1  2 14 15  7 11  3]
[0.303275 0.303275 0.369809 0.369809 0.394438 0.394438 0.399027 0.399027
 0.402213 0.411384 0.448186 0.448186 0.4509   0.45108

In [50]:
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
madmol='HCl'
xc='hf'
operator='excited-state'
def generateDaltonExcitedDataTable(madmol,xc,num_states,basis_list):
    output_files=[]
    for basis in basis_list:
        run_directory,dal_input,mol_input=dalton_excited_input(madmol,xc,basis,num_states)
        d_output_file=dal_input.split('.')[0]+"_"+mol_input.split('.')[0]+'.out'
        d_out,d_error=run_dalton(run_directory,dal_input,mol_input)
        with open(run_directory+'/'+d_output_file) as outfile:
            json_obj=daltonToJson()
            output_files.append(json_obj.convert(outfile))
    
    if not os.path.exists("data_tables"):
        os.mkdir("data_tables")
    
    e,r,ffs=createExcitedTables(output_files,basis_list,num_states)
    e.to_csv('data_tables/'+'e_dalton_'+madmol+'_'+xc+'_'+'excited-state'+'.csv')
    r.to_csv('data_tables/'+'r_dalton_'+madmol+'_'+xc+'_'+'excited-state'+'.csv')
    for b,f in zip(basis_list,ffs):
        f.to_csv('data_tables/'+'r_dalton_'+madmol+'_'+xc+'_'+'excited-state_'+b+'.csv')
        
        
    return e,r,ffs



In [51]:
e,r,ffs=generateDaltonExcitedDataTable('H2O',xc,4,basis_list)

a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.015       0.015

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Singlet electronic excitation energies
clearing no good if seen more than twice
yoou founnd meeeee
                 Sym.   Mode   Frequency    Frequency

                 Sym.   Mode   Frequency    Frequency

                ex. st.  No.      (au)         (eV)

               ----------------------------------------

found ------ appending 
read line: ['1', '1', '0.403345', '10.975570']
read line: ['1', '2', '0.470405', '12.800363']
read line: ['1', '3', '0.539084', '14.669223']
read line: ['1', '4', '0.563839', '15.342847']
read line: ['2', '1', '0.317328', '8.634925']
read line: ['2', '2', '0.444834', '12.104555']
read line: ['2', '3', '0

In [54]:
ffs

[    Sym  Mode      freq
 0     2     1  0.317328
 1     4     1  0.379087
 2     1     1  0.403345
 3     2     2  0.444834
 4     3     1  0.463698
 5     1     2  0.470405
 6     4     2  0.484360
 7     2     3  0.486557
 8     3     2  0.526855
 9     2     4  0.528251
 10    4     3  0.531884
 11    1     3  0.539084
 12    1     4  0.563839
 13    3     3  0.577233
 14    3     4  0.607759
 15    4     4  0.664655,
     Sym  Mode      freq
 0     2     1  0.317867
 1     4     1  0.379305
 2     1     1  0.402030
 3     2     2  0.432478
 4     1     2  0.454686
 5     3     1  0.462318
 6     2     3  0.467034
 7     4     2  0.468226
 8     4     3  0.512236
 9     2     4  0.512822
 10    1     3  0.520669
 11    3     2  0.525037
 12    1     4  0.542575
 13    3     3  0.546009
 14    3     4  0.560939
 15    4     4  0.624390,
     Sym  Mode      freq
 0     2     1  0.317840
 1     4     1  0.379158
 2     1     1  0.401581
 3     2     2  0.425444
 4     1     2  0.44483